In [2]:
import json
from src.model.models.BasicXGBOOST1 import BasicXGBOOST1
from src.features.featurizer import Featurizer2
from src.dataset.dataset2 import Dataset2
import numpy as np
import pandas as pd
import polars as pl

In [17]:
path = 'data/raw/xnas-itch-20180501-20250314.ohlcv-1m.parquet'
data = pl.read_parquet(path)
data = data.sort('ts_event')
data.write_parquet(path)